In [111]:
# https://stackoverflow.com/questions/16236684/apply-pandas-function-to-column-to-create-multiple-new-columns
import random
import nltk
from pprint import pprint
import pandas as pd
from pandas.io.json import json_normalize
# nltk.download('names')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

### Load Data

In [29]:
from nltk.corpus import names
print('Count male names: {}'.format(len(names.words('male.txt'))))
print('Count female names: {}'.format(len(names.words('female.txt'))))

Count male names: 2943
Count female names: 5001


In [81]:
labeled_names = ([(name, 'male') for name in names.words('male.txt')] 
                 + [(name, 'female') for name in names.words('female.txt')])
random.shuffle(labeled_names)
labeled_names = pd.DataFrame(labeled_names, columns=['name', 'label'])
labeled_names.head()

,name,label
0,Raymond,male
1,Vivienne,female
2,Arron,male
3,Jock,male
4,Bret,male


### Feature Engineering

In [106]:
def get_features(name):
    features = {}
#     features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
#     for letter in 'abcdefghijklmnopqrstuvwxyz':
#         features["count({})".format(letter)] = name.lower().count(letter)
#         features["has({})".format(letter)] = (letter in name.lower())
    return features


In [112]:
# featuresets = [(get_features(name), gender) for (name, gender) in labeled_names]
# print(len(featuresets))
# pprint(featuresets[:5])

df = labeled_names.copy()
df['tmp'] = df['name'].apply(lambda x: get_features2(x))
df = pd.concat([df, json_normalize(df['tmp'])], axis=1)
df.drop(['tmp'], inplace=True, axis=1)
df.head()

,name,label,last_letter
0,Raymond,male,d
1,Vivienne,female,e
2,Arron,male,n
3,Jock,male,k
4,Bret,male,t


### Classify

#### Data Split

In [57]:
train_set, dev_set, test_set = (featuresets[1000:], 
                                featuresets[500:1000],
                                featuresets[:500])
print('len train set: {}'.format(len(train_set)))
print('len dev set: {}'.format(len(dev_set)))
print('len test set: {}'.format(len(test_set)))

len train set: 6944
len dev set: 500
len test set: 500


#### Model

In [58]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

#### Evaluate

In [59]:
# test

name = 'Arlo'
name = 'Eileen'
name = 'Laura'
classifier.classify(gender_features(name))

'female'

In [65]:
# Accuracy

nltk.classify.accuracy(classifier, dev_set)

0.744

In [66]:
# Most Informative Features

classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'k'              male : female =     42.2 : 1.0
             last_letter = 'a'            female : male   =     33.8 : 1.0
             last_letter = 'v'              male : female =     17.2 : 1.0
             last_letter = 'f'              male : female =     15.6 : 1.0
             last_letter = 'p'              male : female =     11.6 : 1.0


#### Error Analysis

In [ ]:
errors = []
for (name, tag) in dev_set:
    guess = classifier.classify(gender_features(name))
    if guess != tag:
        errors.append( (tag, guess, name) )